In [1]:
 pip install sklearn-crfsuite

In [2]:
pip install conllu

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
import pandas as pd

In [5]:
DATA = pd.read_csv('/content/drive/My Drive/NLP Assignment-4/hi-ud-train.conllu')


In [6]:

DATA.head()

,ID,WORD,POS_TAG
0,1.0,yaha,DET
1,2.0,eSiyA,PROPN
2,3.0,kI,ADP
3,4.0,sabase,ADV
4,5.0,badZI,ADJ


In [7]:
tst_f = open('/content/drive/My Drive/NLP Assignment-4/hi-ud-test .conllu', 'r') 

In [8]:
tst_data = tst_f.readlines()
tst_lines = list()
tst_set = list()
for l in tst_data:
  li = tuple((l.strip()).split('\t'))
  if li[0]!='':
    tst_lines.append(li)
  if li[0]!='' and li[1]== ".":
    tst_set.append(tst_lines)
    tst_lines = []


In [9]:
tst_set[0]

[('ID', 'WORD', 'TAG'),
 ('1', 'rAmAyaNa', 'PROPN'),
 ('2', 'kAla', 'PROPN'),
 ('3', 'meM', 'ADP'),
 ('4', 'BagavAna', 'NOUN'),
 ('5', 'rAma', 'PROPN'),
 ('6', 'ke', 'ADP'),
 ('7', 'puwra', 'NOUN'),
 ('8', 'kuSa', 'PROPN'),
 ('9', 'kI', 'ADP'),
 ('10', 'rAjaXAnI', 'NOUN'),
 ('11', 'kuSAvawI', 'PROPN'),
 ('12', 'ko', 'ADP'),
 ('13', '483', 'PROPN'),
 ('14', 'IsA', 'PROPN'),
 ('15', 'pUrva', 'PROPN'),
 ('16', 'buxXa', 'PROPN'),
 ('17', 'ne', 'ADP'),
 ('18', 'apane', 'PRON'),
 ('19', 'aMwima', 'ADJ'),
 ('20', 'viSrAma', 'NOUN'),
 ('21', 'ke', 'ADP'),
 ('22', 'lie', 'ADP'),
 ('23', 'cunA', 'VERB'),
 ('24', '.', 'PUNCT')]

In [10]:
DATA = DATA.dropna()
DATA= DATA.reset_index()

In [11]:
trn_lines = []
trn_set = []
for i in range(len(DATA)):
  id = str(DATA['ID'][i])
  word = DATA['WORD'][i]
  ptag = DATA['POS_TAG'][i]
  trn_lines.append((id,word,ptag))
  if word ==".":
    trn_set.append(trn_lines[1:])
    trn_lines=[]

In [12]:
trn_set[1]

[('2.0', 'navAba', 'NOUN'),
 ('3.0', 'SAhajehana', 'PROPN'),
 ('4.0', 'ne', 'ADP'),
 ('5.0', 'banavAyA', 'VERB'),
 ('6.0', 'WA', 'AUX'),
 ('7.0', '.', 'PUNCT')]

In [13]:
import nltk
import sklearn
import scipy.stats
import sklearn_crfsuite
from itertools import chain
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [24]:
def word_2_feat(sentence, i):
    word = sentence[i][1]
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-4:]': word[-4:],
        'word[-3:]': word[-3:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),      
    }
    if i > 0:
        word1 = sentence[i-1][1]
        postag1 = sentence[i-1][2]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
        })
    else:
        features['BOS'] = True
        
    if i < len(sentence)-1:
        word1 = sentence[i+1][1]
        postag1 = sentence[i+1][2]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
        })
    else:
        features['EOS'] = True
                
    return features


def sentence_2_feat(sentence):
    return [word_2_feat(sentence, i) for i in range(len(sentence))]

def sentence_2_label(sentence):
    return [label for id, token, label in sentence]

def sentence_2_tok(sentence):
    return [token for id,token, label in sentence]

Feature description:

* Lowercasing (previous, current and next words)
* Checking for uppercase
* Checking for title
* Checking for digit
* Checking for later part of words(last 3/2 letters)


In [25]:
%%time
X_trn = [sentence_2_feat(i) for i in trn_set]
y_trn = [sentence_2_label(i) for i in trn_set]

X_tst = [sentence_2_feat(i) for i in tst_set]
y_tst = [sentence_2_label(i) for i in tst_set]

CPU times: user 30.7 ms, sys: 2.12 ms, total: 32.8 ms
Wall time: 35.2 ms


In [26]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_trn, y_trn)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [27]:
labels = list(crf.classes_)
# labels.remove('O')
labels

['PROPN',
 'ADP',
 'ADV',
 'ADJ',
 'NOUN',
 'NUM',
 'AUX',
 'PUNCT',
 'VERB',
 'CCONJ',
 'DET',
 'PRON',
 'PART',
 'SCONJ',
 'X']

In [28]:
y_pred = crf.predict(X_tst)
metrics.flat_f1_score(y_tst, y_pred,
                      average='weighted', labels=labels)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


0.8283968426555907

In [29]:
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_tst, y_pred, labels=sorted_labels, digits=3
))

              precision    recall  f1-score   support

           X      0.000     0.000     0.000         0
        PART      0.941     0.970     0.955        33
       CCONJ      1.000     1.000     1.000        25
       SCONJ      0.667     0.667     0.667         3
         ADJ      0.617     0.755     0.679        94
         ADP      0.962     0.971     0.966       309
         ADV      0.667     0.381     0.485        21
        VERB      0.802     0.778     0.790        99
         DET      0.886     0.861     0.873        36
        NOUN      0.740     0.863     0.797       329
        PRON      0.871     0.831     0.850        65
       PROPN      0.625     0.448     0.522       145
         NUM      1.000     0.880     0.936        25
       PUNCT      1.000     0.830     0.907       135
         AUX      0.909     0.935     0.922       139

   micro avg      0.831     0.832     0.832      1458
   macro avg      0.779     0.745     0.757      1458
weighted avg      0.834   

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [30]:
from collections import Counter

def prnt_trans(trans_feat):
    for (label_from, label_to), weight in trans_feat:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("10 most common transition features:")
prnt_trans(Counter(crf.transition_features_).most_common(10))

print("\n 10 least common transition features")
prnt_trans(Counter(crf.transition_features_).most_common()[-10:])

10 most common transition features:
VERB   -> AUX     3.925082
PROPN  -> PROPN   2.860289
AUX    -> AUX     2.337864
VERB   -> SCONJ   2.154873
ADJ    -> NOUN    2.142769
AUX    -> SCONJ   2.136516
PROPN  -> ADP     1.992648
NUM    -> NOUN    1.697616
PROPN  -> CCONJ   1.660574
DET    -> ADJ     1.641883

 10 least common transition features
PROPN  -> AUX     -1.121067
ADV    -> AUX     -1.145800
ADV    -> PROPN   -1.176615
AUX    -> ADJ     -1.241653
AUX    -> VERB    -1.257049
CCONJ  -> AUX     -1.488535
AUX    -> ADP     -1.642774
ADJ    -> PRON    -1.698902
ADJ    -> ADP     -2.239316
DET    -> ADP     -2.253842


In [31]:
corr_pred = 0
x=0
for i in range(len(y_pred)):
  for j in range(len(y_pred[i])):
    if(y_pred[i][j]==y_tst[i][j]):
      corr_pred = corr_pred+1
    x+=1
print("FINAL TEST ACCURACY : ",corr_pred/x)


FINAL TEST ACCURACY :  0.8313913639479096


In [22]:
sorted_labels = sorted(
    labels, 
    key=lambda name: (name[1:], name[0])
)
y_pred1 = crf.predict(X_trn)
print(metrics.flat_classification_report(
    y_trn, y_pred1, labels=sorted_labels, digits=3
))

              precision    recall  f1-score   support

           X      1.000     1.000     1.000         2
        PART      1.000     1.000     1.000       160
       CCONJ      0.993     1.000     0.996       138
       SCONJ      0.980     1.000     0.990        49
         ADJ      0.998     1.000     0.999       539
         ADP      0.999     0.999     0.999      1387
         ADV      0.989     0.979     0.984        96
        VERB      0.998     0.981     0.990       634
         DET      1.000     1.000     1.000       157
        NOUN      0.995     0.999     0.997      1529
        PRON      1.000     0.996     0.998       278
       PROPN      1.000     0.989     0.994       611
         NUM      1.000     1.000     1.000       147
       PUNCT      1.000     1.000     1.000       564
         AUX      0.985     1.000     0.993       730

    accuracy                          0.996      7021
   macro avg      0.996     0.996     0.996      7021
weighted avg      0.996   

In [23]:
corr_pred = 0
x=0
for i in range(len(y_pred1)):
  for j in range(len(y_pred1[i])):
    if(y_pred1[i][j]==y_trn[i][j]):
      corr_pred = corr_pred+1
    x+=1
print("FINAL TRAIN ACCURACY : ",corr_pred/x)


FINAL TRAIN ACCURACY :  0.9962968238142714
